In [69]:
%cd /home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger-detection/BGN-ST

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger-detection/BGN-ST


In [70]:
import torch
import pickle
from dataloaders import get_data_loaders
import matplotlib.pyplot as plt
from utils.log import load_checkpoint
from collections import defaultdict
from itertools import islice
from tqdm.notebook import tqdm
from models.Bipartite_Attention_gLRI import Bipartite_Attention as Model
import numpy as np
import matplotlib.colors as colors

In [71]:
# α=0.3
config_file_path = 'train_results/biatt-glri/experiment_2023-02-11_03:56:54/config.pkl'
#config_file_path = 'train_results/biatt-glri/experiment_2023-02-12_08:24:52/config.pkl'
with open(config_file_path, 'rb') as f:
    config = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'train_results/biatt-glri/experiment_2023-02-11_03:56:54/config.pkl'

In [ ]:
config['lri']['sigma']

In [ ]:
config['lri']['beta']

In [ ]:
dconfig = config['data']
dconfig['use_trigger'] = True
dconfig['use_nontrigger'] = False
train_data, val_data, test_data = get_data_loaders(**dconfig)

In [ ]:
DEVICE = 'cuda:0'
# α=0.3
checkpoint_file = 'train_results/biatt-glri/experiment_2023-02-11_03:56:54/checkpoints/model_checkpoint_012.pth.tar'
#checkpoint_file = 'train_results/biatt-glri/experiment_2023-02-12_08:24:52/checkpoints/model_checkpoint_011.pth.tar'
mconfig = config['model']
model = Model(**mconfig)
model = load_checkpoint(checkpoint_file, model)
model = model.to(DEVICE)
model.eval()

# Radius Change Analysis

In [ ]:
tracks = []
sigmas = []
good_hits = []
tt_tracks = []
n_tracks = []
momentums = []

In [ ]:
N_BATCHES = len(val_data)
for batch in tqdm(islice(val_data, 0, N_BATCHES), total=N_BATCHES):
    mask = torch.zeros(batch.track_vector.shape[:-1])
    for i, n in enumerate(batch.n_tracks):
        mask[i, :n] = 1
        
    mask = mask.to(DEVICE)
    track_vector = batch.track_vector.to(DEVICE)
    is_trigger_track = batch.is_trigger_track.to(DEVICE, torch.bool)
    trigger = (batch.trigger.to(DEVICE) == 1).unsqueeze(-1)
    
    _, pred_sigmas, valid_hits = model(track_vector, mask)
    tracks.extend(track_vector.detach().cpu().numpy())
    sigmas.extend(pred_sigmas.detach().cpu().numpy())
    good_hits.extend(valid_hits.detach().cpu().numpy())
    tt_tracks.extend(is_trigger_track.unsqueeze(-1).repeat(1, 1, 5).detach().cpu().numpy())
    n_tracks.extend(batch.n_tracks.detach().cpu().numpy())
    momentums.extend(batch.momentums.detach().cpu().numpy())

In [ ]:
tracks[3].shape

In [ ]:
N_BATCHES = len(val_data)
orig_radii = []
new_radii = []
is_trigger = []
for batch in tqdm(islice(val_data, 0, N_BATCHES), total=N_BATCHES):
    mask = torch.zeros(batch.track_vector.shape[:-1])
    for i, n in enumerate(batch.n_tracks):
        mask[i, :n] = 1
        
    mask = mask.to(DEVICE)
    track_vector = batch.track_vector.to(DEVICE)
    is_trigger_track = batch.is_trigger_track.to(DEVICE, torch.bool)
    trigger = (batch.trigger.to(DEVICE) == 1).unsqueeze(-1)
    
    r_orig, r_new = model.get_radii(track_vector, mask)
    mask = mask.to(torch.bool)
    orig_radii.append(r_orig[mask].detach().cpu().numpy())
    new_radii.append(r_new[mask].detach().cpu().numpy())
    is_trigger.append(is_trigger_track[mask].detach().cpu().numpy())
orig_radii = np.concatenate(orig_radii, axis=0)
new_radii = np.concatenate(new_radii, axis=0)
is_trigger = np.concatenate(is_trigger, axis=0)

In [ ]:
_ = plt.hist(new_radii[new_radii < 25], bins=500)

In [ ]:
mask = (orig_radii > 0) & (orig_radii < 25) & is_trigger
diff = (new_radii[mask] - orig_radii[mask]) / orig_radii[mask]
_ = plt.hist(100*diff[np.abs(diff) < 4], bins=500)
plt.xlabel('% Difference')
plt.ylabel('Count')
plt.title('Trigger Tracks % Radius Change (Cutoff = 400%)')

In [ ]:
mask = (orig_radii > 0) & (orig_radii < 25)
diff = (new_radii[mask] - orig_radii[mask]) / orig_radii[mask]
_ = plt.hist(100*diff[np.abs(diff) < 4], bins=500)
plt.xlabel('% Difference')
plt.ylabel('Count')
plt.title('All Tracks % Radius Change (Cutoff = 400%)')

In [72]:
mask = (orig_radii > 0) & (orig_radii < 25)
diff = (new_radii[mask] - orig_radii[mask]) / orig_radii[mask]
_ = plt.hist(diff, bins=500)
plt.yscale('log')
plt.xlabel('% Difference')
plt.xscale('log')
plt.ylabel('Count')
plt.title('All Tracks % Radius Change')

NameError: name 'orig_radii' is not defined

In [73]:
mask = (orig_radii > 0) & (orig_radii < 25) & is_trigger
diff = (new_radii[mask] - orig_radii[mask]) / orig_radii[mask]
_ = plt.hist(diff, bins=500)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('% Difference')
plt.ylabel('Count')
plt.title('Trigger Tracks % Radius Change')

NameError: name 'orig_radii' is not defined

In [10]:
plt.hist(orig_radii[orig_radii < 25], bins=500)
plt.title('All Tracks Radius Distribution (Cutoff = 25)')
plt.xlabel('Radius')
plt.ylabel('Count')

In [11]:
plt.hist(orig_radii[(orig_radii < 25) & is_trigger], bins=500)
plt.title('Trigger Tracks Radius Distribution (Cutoff = 25)')
plt.xlabel('Radius')
plt.ylabel('Count')

In [12]:
plt.hist(orig_radii, bins=500)
plt.title('All Tracks Radius Distribution')
plt.xlabel('Radius')
plt.ylabel('Count')
plt.xscale('log')
plt.yscale('log')

In [13]:
plt.hist(orig_radii[is_trigger], bins=500)
plt.title('Trigger Tracks Radius Distribution')
plt.xlabel('Radius')
plt.ylabel('Count')
plt.xscale('log')
plt.yscale('log')

# Determinant Analysis

In [14]:
ranks = []
for track, sigma, hits_mask, trigger_mask in zip(tracks, sigmas, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask*trigger_mask
    all_hits = track[..., :15].reshape(-1, 3)
    sigma = sigma.reshape(-1, 3, 3)
    dets = np.linalg.det(sigma)
    ranks.append(np.argsort(dets[mask])/np.sum(hits_mask))
ranks = np.concatenate(ranks, axis=0)

In [15]:
trigger_determinants = []
for track, sigma, hits_mask, trigger_mask in zip(tracks, sigmas, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask * trigger_mask
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    trigger_determinants.append(np.linalg.det(sigma))
trigger_determinants = np.concatenate(trigger_determinants, axis=0)

In [16]:
nontrigger_determinants = []
for track, sigma, hits_mask, trigger_mask in zip(tracks, sigmas, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask * (1-trigger_mask)
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    nontrigger_determinants.append(np.linalg.det(sigma))
nontrigger_determinants = np.concatenate(nontrigger_determinants, axis=0)

In [17]:
_ = plt.hist(ranks, bins=100)
plt.xlabel('Hit Determinant Rank')
plt.ylabel('Count')
plt.title('Rank of Trigger Track Hit Determinant')
plt.yscale('log')

In [18]:
_ = plt.hist(ranks, bins=100, cumulative=True)
plt.xlabel('Hit Determinant Rank')
plt.ylabel('Count')
plt.title('Rank of Trigger Track Hit Determinant (Cumulative)')

In [19]:
_ = plt.hist(nontrigger_determinants, bins=100, label='Non Trigger')
_ = plt.hist(trigger_determinants, bins=100, label='Trigger')
plt.xlabel('Hit Determinant')
plt.ylabel('Count')
plt.title('Track Hit Determinant')
plt.legend()
plt.yscale('log')

In [20]:
_ = plt.hist(nontrigger_determinants, bins=100, label='Non Trigger', cumulative=True)
_ = plt.hist(trigger_determinants, bins=100, label='Trigger', cumulative=True)
plt.xlabel('Hit Determinant')
plt.ylabel('Count')
plt.title('Track Hit Determinant')
plt.legend()

In [21]:
def f(x, sigma):
    #sigma = sigma[:2, :2]
    coeff = 1/np.sqrt(2*np.pi*np.linalg.det(sigma))
    sigma_i = np.linalg.inv(sigma)
    xp = np.sum(x * np.einsum('...ij,...j->...i', sigma_i, x), axis=-1)
    return coeff*np.exp(-1/2*xp)

def f(x, sigma):
    #sigma = sigma[:2, :2]
    coeff = 1/np.sqrt(2*np.pi*np.linalg.det(sigma))
    sigma_i = np.linalg.inv(sigma)
    xp = np.sum(x * np.einsum('...ij,...j->...i', sigma_i, x), axis=-1)
    return -1/2 * xp

In [22]:
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x, y)
p = np.stack([X, Y, np.zeros(X.shape)], axis=-1)
levels = np.linspace(-1, 0, 100)

In [23]:
for track, sigma, hits_mask, trigger_mask in islice(zip(tracks, sigmas, good_hits, tt_tracks), 2, 10):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask #* trigger_mask
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    plt.figure(figsize=(15, 15))
    plt.scatter(all_hits[:, 0], all_hits[:, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    for hit, s in zip(hits, sigma):
        prob = f(p, 10*s)
        #X_t = X[prob > 0.01]
        #Y_t = Y[prob > 0.01]
        #prob = prob[prob > 0.01]
        plt.contour(X + hit[0], Y + hit[1], prob, levels=levels)
        
        
        
        
    #plt.figure()
    #plt.scatter(hits[:, 0], hits[:, 1])
    #print(sigma)

In [24]:
for track, sigma, hits_mask, trigger_mask in islice(zip(tracks, sigmas, good_hits, tt_tracks), 2, 3):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask * trigger_mask
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    plt.figure(figsize=(15, 15))
    plt.scatter(all_hits[:, 0], all_hits[:, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    for hit, s in zip(hits, sigma):
        prob = f(p, s*100)
        #X_t = X[prob > 0.01]
        #Y_t = Y[prob > 0.01]
        #prob = prob[prob > 0.01]
        plt.contour(X + hit[0], Y + hit[1], prob, levels)
        
        
        
        
    #plt.figure()
    #plt.scatter(hits[:, 0], hits[:, 1])
    #print(sigma)

# Covariance Matrix Eigenvalue Analysis

In [25]:
alignments = []
symmetricities = []
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    #print(sigma.shape)
    m = np.repeat(np.expand_dims(momentum, 1), 5, axis=1).reshape(-1, 3)
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask * trigger_mask
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    m = m[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    m /= np.expand_dims(np.linalg.norm(m, axis=-1), -1)
    alignment = np.einsum('hf,hfi->hi', m, eigenvectors)
    preferred_direction = np.expand_dims(np.argmax(eigenvalues, axis=-1), -1)
    alignments.append(np.abs(np.take_along_axis(alignment, preferred_direction, -1)))
    symmetricities.append(np.prod(eigenvalues/np.expand_dims(np.max(eigenvalues, axis=-1), -1), -1))
trigger_alignments = np.concatenate(alignments, axis=0)
trigger_symmetricities = np.concatenate(symmetricities, axis=0)
plt.figure(figsize=(10, 10))
h = plt.hist2d(np.squeeze(trigger_symmetricities), np.squeeze(trigger_alignments), bins=50)#, norm=colors.LogNorm())
plt.colorbar(h[3])
plt.xlabel('Symmetricity')
plt.ylabel('Alignment')
plt.title('Histogram of Trigger Symmetricites and Alignments')

In [26]:
alignments = []
symmetricities = []
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    m = np.repeat(np.expand_dims(momentum, 1), 5, axis=1).reshape(-1, 3)
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask
    all_hits = track[..., :15].reshape(-1, 3)
    hits = all_hits[mask]
    m = m[mask]
    sigma = sigma.reshape(-1, 3, 3)[mask]
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    m /= np.expand_dims(np.linalg.norm(m, axis=-1), -1)
    alignment = np.einsum('hf,hfi->hi', m, eigenvectors)
    preferred_direction = np.expand_dims(np.argmax(eigenvalues, axis=-1), -1)
    alignments.append(np.abs(np.take_along_axis(alignment, preferred_direction, -1)))
    symmetricities.append(np.prod(eigenvalues/np.expand_dims(np.max(eigenvalues, axis=-1), -1), -1))
all_alignments = np.concatenate(alignments, axis=0)
all_symmetricities = np.concatenate(symmetricities, axis=0)
plt.figure(figsize=(10, 10))
h = plt.hist2d(np.squeeze(all_symmetricities), np.squeeze(all_alignments), bins=50)#, norm=colors.LogNorm())
plt.colorbar(h[3])
plt.xlabel('Symmetricity')
plt.ylabel('Alignment')
plt.title('Histogram of All Symmetricites and Alignments')

## First Mode

### All Tracks

In [27]:
all_eigenvalues = []
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask
    sigma = sigma.reshape(-1, 3, 3)[mask]
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    eigenvalues = np.sort(eigenvalues, axis=-1)
    all_eigenvalues.append(eigenvalues)
all_eigenvalues = np.concatenate(all_eigenvalues, axis=0)

In [28]:
layer_eigenvalues = defaultdict(list)
layer_symmetricities = defaultdict(list)
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    for layer in range(5):
        layer_hits_mask = hits_mask[:, layer:layer+1]
        layer_sigma = sigma[:, layer:layer+1, :, :]
        
        layer_hits_mask = layer_hits_mask.reshape(-1)
        mask = layer_hits_mask #* trigger_mask[..., ::5].reshape(-1)
        layer_sigma = layer_sigma.reshape(-1, 3, 3)[mask]
        eigenvalues, eigenvectors = np.linalg.eig(layer_sigma)
        eigenvalues = np.sort(eigenvalues, axis=-1)
        layer_eigenvalues[layer].append(eigenvalues)
        layer_symmetricities[layer].append(np.prod(eigenvalues/np.expand_dims(np.max(eigenvalues, axis=-1), -1), -1))
        
for k in layer_eigenvalues.keys():
    layer_eigenvalues[k] = np.concatenate(layer_eigenvalues[k], axis=0)
    layer_symmetricities[k] = np.concatenate(layer_symmetricities[k], axis=0)

In [29]:
plt.title("All Tracks Distribution of Symmetricity")
plt.xlabel('Symmetricity')
plt.ylabel('Count')
s = plt.hist(all_symmetricities[:N], bins=50)
s_bins = s[1]

In [30]:
N = all_eigenvalues.shape[0]
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(all_symmetricities[:N], all_eigenvalues[:N, 2]/all_eigenvalues[:N, 1])
plt.xscale('log')
plt.yscale('log')
l1 = all_eigenvalues[:N, 2]
l2 = all_eigenvalues[:N, 1]
l3 = all_eigenvalues[:N, 0]
#all_indices = np.arange(N)
#cutoff_index = np.argmax(l2[all_symmetricities[:N] <= s_bins[1]]/l3[all_symmetricities[:N] <= s_bins[1]])
MODE_1_CUTOFF = np.min(l1[all_symmetricities[:N] <= s_bins[1]]/l2[all_symmetricities[:N] <= s_bins[1]])
plt.plot([s_bins[1], s_bins[1]], [plt.ylim()[0], MODE_1_CUTOFF], color='black')
plt.plot([plt.xlim()[0], s_bins[1]], [MODE_1_CUTOFF, MODE_1_CUTOFF], color='black')
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$ (Cutoff={MODE_1_CUTOFF})')

In [31]:
for i in range(5):
    N = layer_eigenvalues[i].shape[0]
    l1 = layer_eigenvalues[i][:N, 2]
    l2 = layer_eigenvalues[i][:N, 1]
    l3 = layer_eigenvalues[i][:N, 0]
    plt.scatter((l1/l2)[layer_symmetricities[i][:N] <= s_bins[1]], (l2/l3)[layer_symmetricities[i][:N] <= s_bins[1]], label=f'Layer {i}')

plt.legend()
plt.title(f'All Tracks $\lambda_1/\lambda_2$ vs $\lambda_2/\lambda_3$')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.xscale('log')

In [32]:
N = all_eigenvalues.shape[0]
l1 = all_eigenvalues[:N, 2]
l2 = all_eigenvalues[:N, 1]
l3 = all_eigenvalues[:N, 0]
plt.title('All Tracks \lambda_1/\lambda_2$ vs $\lambda_2/\lambda_3$ for First Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.scatter((l1/l2)[all_symmetricities[:N] <= s_bins[1]], (l2/l3)[all_symmetricities[:N] <= s_bins[1]])
plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

In [33]:
tt_tracks[0].shape

In [34]:
N = all_eigenvalues.shape[0]
l1 = all_eigenvalues[:N, 2]
l2 = all_eigenvalues[:N, 1]
l3 = all_eigenvalues[:N, 0]
plt.title('All Tracks Distribution of ($\lambda_1/\lambda_2$, $\lambda_2/\lambda_3$) for First Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
h = plt.hist2d((l1/l2)[all_symmetricities[:N] <= s_bins[1]], (l2/l3)[all_symmetricities[:N] <= s_bins[1]], bins=20)
plt.colorbar(h[3])
#plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

### Trigger Tracks

In [35]:
trigger_eigenvalues = []
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask * trigger_mask
    sigma = sigma.reshape(-1, 3, 3)[mask]
    eigenvalues, eigenvectors = np.linalg.eig(sigma)
    eigenvalues = np.sort(eigenvalues, axis=-1)
    trigger_eigenvalues.append(eigenvalues)
trigger_eigenvalues = np.concatenate(trigger_eigenvalues, axis=0)

In [36]:
plt.title("Trigger Tracks Distribution of Symmetricity")
plt.xlabel('Symmetricity')
plt.ylabel('Count')
st = plt.hist(trigger_symmetricities[:N], bins=50)
st_bins = s[1]

In [37]:
N = trigger_eigenvalues.shape[0]
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(trigger_symmetricities[:N], trigger_eigenvalues[:N, 2]/trigger_eigenvalues[:N, 1])
plt.xscale('log')
plt.yscale('log')
l1 = trigger_eigenvalues[:N, 2]
l2 = trigger_eigenvalues[:N, 1]
l3 = trigger_eigenvalues[:N, 0]
#trigger_indices = np.arange(N)
#cutoff_index = np.argmax(l2[trigger_symmetricities[:N] <= s_bins[1]]/l3[trigger_symmetricities[:N] <= s_bins[1]])
plt.plot([s_bins[1], s_bins[1]], [plt.ylim()[0], MODE_1_CUTOFF], color='black')
plt.plot([plt.xlim()[0], s_bins[1]], [MODE_1_CUTOFF, MODE_1_CUTOFF], color='black')
plt.title(f'Trigger Tracks Symmetricity vs $\lambda_1/\lambda_2$ (Cutoff={MODE_1_CUTOFF})')

In [38]:
N = trigger_eigenvalues.shape[0]
l1 = trigger_eigenvalues[:N, 2]
l2 = trigger_eigenvalues[:N, 1]
l3 = trigger_eigenvalues[:N, 0]
plt.title('Trigger Tracks $\lambda_1/\lambda_2$ vs $\lambda_2/\lambda_3$ for First Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.scatter((l1/l2)[trigger_symmetricities[:N] <= s_bins[1]], (l2/l3)[trigger_symmetricities[:N] <= s_bins[1]])
plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

## Second Mode

In [39]:
SECOND_MODE_START, SECOND_MODE_END = s[1][15], s[1][20]

In [40]:
FIRST_MODE_END

In [41]:
SECOND_MODE_START, SECOND_MODE_END

In [42]:
THIRD_MODE_START, THIRD_MODE_END

In [43]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= SECOND_MODE_START) & (all_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(all_symmetricities[:N][mask], all_eigenvalues[:N, 2][mask]/all_eigenvalues[:N, 1][mask])
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$')

In [44]:
for i in range(5):
    N = layer_eigenvalues[i].shape[0]
    mask = (layer_symmetricities[i][:N] >= SECOND_MODE_START) & (layer_symmetricities[i][:N] < SECOND_MODE_END)


    plt.scatter(layer_symmetricities[i][:N][mask], layer_eigenvalues[i][:N, 2][mask]/layer_eigenvalues[i][:N, 1][mask], label=f'Layer {i}')

plt.legend(loc='upper right')
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$')
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')

In [45]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= SECOND_MODE_START) & (all_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
h = plt.hist2d(all_symmetricities[:N][mask], all_eigenvalues[:N, 2][mask]/all_eigenvalues[:N, 1][mask], bins=20)
plt.colorbar(h[3])
plt.title(f'All Tracks Distribution of (Symmetricity, $\lambda_1/\lambda_2$)')

In [46]:
N = all_eigenvalues.shape[0] // 10
mask = (all_symmetricities[:N] >= SECOND_MODE_START) & (all_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.scatter(all_symmetricities[:N][mask], all_eigenvalues[:N, 1][mask]/all_eigenvalues[:N, 0][mask])
plt.title(f'All Tracks Symmetricity vs $\lambda_2/\lambda_3$')

In [47]:
for i in range(5):
    N = layer_eigenvalues[i].shape[0]
    mask = (layer_symmetricities[i][:N] >= SECOND_MODE_START) & (layer_symmetricities[i][:N] < SECOND_MODE_END)


    plt.scatter(layer_symmetricities[i][:N][mask], layer_eigenvalues[i][:N, 1][mask]/layer_eigenvalues[i][:N, 0][mask], label=f'Layer {i}')

plt.legend(loc='upper right')
plt.title(f'All Tracks Symmetricity vs $\lambda_2/\lambda_3$')
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')

In [48]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= SECOND_MODE_START) & (all_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
h = plt.hist2d(all_symmetricities[:N][mask], all_eigenvalues[:N, 1][mask]/all_eigenvalues[:N, 0][mask], bins=20)
plt.colorbar(h[3])
plt.title(f'All Tracks Distribution of (Symmetricity, $\lambda_2/\lambda_3$)')

In [49]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= SECOND_MODE_START) & (all_symmetricities[:N] < SECOND_MODE_END)

l1 = all_eigenvalues[:N, 2][mask]
l2 = all_eigenvalues[:N, 1][mask]
l3 = all_eigenvalues[:N, 0][mask]
plt.title('All Tracks Distribution of ($\lambda_1/\lambda_2$, $\lambda_2/\lambda_3$) for Second Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
h = plt.hist2d((l1/l2), (l2/l3), bins=20)
plt.colorbar(h[3])
#plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

## Trigger Tracks

In [50]:
N = trigger_eigenvalues.shape[0]
mask = (trigger_symmetricities[:N] >= SECOND_MODE_START) & (trigger_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(trigger_symmetricities[:N][mask], trigger_eigenvalues[:N, 2][mask]/trigger_eigenvalues[:N, 1][mask])
plt.title(f'Trigger Tracks Symmetricity vs $\lambda_1/\lambda_2$')

In [51]:
N = trigger_eigenvalues.shape[0]
mask = (trigger_symmetricities[:N] >= SECOND_MODE_START) & (trigger_symmetricities[:N] < SECOND_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.scatter(trigger_symmetricities[:N][mask], trigger_eigenvalues[:N, 1][mask]/trigger_eigenvalues[:N, 0][mask])
plt.title(f'Trigger Tracks Symmetricity vs $\lambda_2/\lambda_3$')

In [52]:
N = trigger_eigenvalues.shape[0]
mask = (trigger_symmetricities[:N] >= SECOND_MODE_START) & (trigger_symmetricities[:N] < SECOND_MODE_END)

l1 = trigger_eigenvalues[:N, 2][mask]
l2 = trigger_eigenvalues[:N, 1][mask]
l3 = trigger_eigenvalues[:N, 0][mask]
plt.title('Trigger Tracks Distribution of ($\lambda_1/\lambda_2$, $\lambda_2/\lambda_3$) for Second Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
h = plt.hist2d((l1/l2), (l2/l3), bins=20)
plt.colorbar(h[3])
#plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

## Third Mode

In [53]:
THIRD_MODE_START, THIRD_MODE_END = s[1][35], s[1][40]

In [54]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= THIRD_MODE_START) & (all_symmetricities[:N] < THIRD_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(all_symmetricities[:N][mask], all_eigenvalues[:N, 2][mask]/all_eigenvalues[:N, 1][mask])
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$')

In [55]:
for i in range(5):
    N = layer_eigenvalues[i].shape[0]
    mask = (layer_symmetricities[i][:N] >= THIRD_MODE_START) & (layer_symmetricities[i][:N] < THIRD_MODE_END)


    plt.scatter(layer_symmetricities[i][:N][mask], layer_eigenvalues[i][:N, 2][mask]/layer_eigenvalues[i][:N, 1][mask], label=f'Layer {i}')

plt.legend(loc='upper right')
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$')
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')

In [56]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= THIRD_MODE_START) & (all_symmetricities[:N] < THIRD_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
h = plt.hist2d(all_symmetricities[:N][mask], all_eigenvalues[:N, 2][mask]/all_eigenvalues[:N, 1][mask], bins=20)
plt.colorbar(h[3])
plt.title(f'All Tracks Distribution of (Symmetricity, $\lambda_1/\lambda_2$)')

In [57]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= THIRD_MODE_START) & (all_symmetricities[:N] < THIRD_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_2/\lambda_3$')
plt.scatter(all_symmetricities[:N][mask], all_eigenvalues[:N, 1][mask]/all_eigenvalues[:N, 0][mask])
plt.title(f'All Tracks Symmetricity vs $\lambda_2/\lambda_3$')

In [58]:
for i in range(5):
    N = layer_eigenvalues[i].shape[0]
    mask = (layer_symmetricities[i][:N] >= THIRD_MODE_START) & (layer_symmetricities[i][:N] < THIRD_MODE_END)


    plt.scatter(layer_symmetricities[i][:N][mask], layer_eigenvalues[i][:N, 1][mask]/layer_eigenvalues[i][:N, 0][mask], label=f'Layer {i}')

plt.legend(loc='center left')
plt.title(f'All Tracks Symmetricity vs $\lambda_3/\lambda_3$')
plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_3/\lambda_2$')

In [59]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= THIRD_MODE_START) & (all_symmetricities[:N] < THIRD_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
h = plt.hist2d(all_symmetricities[:N][mask], all_eigenvalues[:N, 1][mask]/all_eigenvalues[:N, 0][mask], bins=20)
plt.colorbar(h[3])
plt.title(f'All Tracks Distribution of (Symmetricity, $\lambda_2/\lambda_3$)')

In [60]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= THIRD_MODE_START) & (all_symmetricities[:N] < THIRD_MODE_END)

l1 = all_eigenvalues[:N, 2][mask]
l2 = all_eigenvalues[:N, 1][mask]
l3 = all_eigenvalues[:N, 0][mask]
plt.title('All Tracks Distribution of ($\lambda_1/\lambda_2$, $\lambda_2/\lambda_3$) for Third Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
h = plt.hist2d((l1/l2), (l2/l3), bins=20)
plt.colorbar(h[3])
#plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

## Fourth Mode

In [61]:
FOURTH_MODE_START, FOURTH_MODE_END = s[1][48], s[1][50]

In [62]:
N = all_eigenvalues.shape[0] // 10
mask = (all_symmetricities[:N] >= FOURTH_MODE_START) & (all_symmetricities[:N] <= FOURTH_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
plt.scatter(all_symmetricities[:N][mask], all_eigenvalues[:N, 2][mask]/all_eigenvalues[:N, 1][mask])
plt.title(f'All Tracks Symmetricity vs $\lambda_1/\lambda_2$')

In [63]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= FOURTH_MODE_START) & (all_symmetricities[:N] <= FOURTH_MODE_END)

plt.xlabel('Symmetricity')
plt.ylabel('$\lambda_1/\lambda_2$')
h = plt.hist2d(all_symmetricities[:N][mask], all_eigenvalues[:N, 1][mask]/all_eigenvalues[:N, 0][mask], bins=20)
plt.colorbar(h[3])
plt.title(f'All Tracks Distribution of (Symmetricity, $\lambda_2/\lambda_3$)')

In [64]:
N = all_eigenvalues.shape[0]
mask = (all_symmetricities[:N] >= FOURTH_MODE_START) & (all_symmetricities[:N] <= FOURTH_MODE_END)

l1 = all_eigenvalues[:N, 2][mask]
l2 = all_eigenvalues[:N, 1][mask]
l3 = all_eigenvalues[:N, 0][mask]
plt.title('All Tracks Distribution of ($\lambda_1/\lambda_2$, $\lambda_2/\lambda_3$) for Fourth Mode')
plt.xlabel('$\lambda_1/\lambda_2$')
plt.ylabel('$\lambda_2/\lambda_3$')
h = plt.hist2d((l1/l2), (l2/l3), bins=20)
plt.colorbar(h[3])
#plt.xscale('log')
#plt.plot((MODE_1_CUTOFF, MODE_1_CUTOFF), plt.ylim())

## Momentum Analysis

In [65]:
all_momentums = []
for track, sigma, momentum, hits_mask, trigger_mask in zip(tracks, sigmas, momentums, good_hits, tt_tracks):
    hits_mask = hits_mask.reshape(-1)
    trigger_mask = trigger_mask.reshape(-1)
    mask = hits_mask[::5]
    all_momentums.append(momentum[mask])
all_momentums = np.concatenate(all_momentums, axis=0)

In [66]:
p_t = np.sqrt(all_momentums[:, 0]**2 + all_momentums[:, 1]**2)
p_z = all_momentums[:, 1]

In [67]:
#plt.hist(p_t/p_z)

In [68]:
plt.hist2d(p_t, p_z, norm=colors.LogNorm())

In [ ]:
e_v.shape

In [ ]:
N_SAMPLES = 10000
U = np.random.uniform(size=(3, 3))
a1, a2 = np.random.uniform(size=(2, 1))
s1, s2 = np.random.normal(size=(2, 3, N_SAMPLES))
e_v = np.sqrt(a1) * U @ s1 + np.sqrt(a2) * np.identity(3) @ s2
e_v = e_v.T
fig, axs = plt.subplots(3, 1, figsize=(20, 60))

_ = axs[0].hist2d(e_v[:, 0], e_v[:, 1])
_ = axs[1].hist2d(e_v[:, 0], e_v[:, 2])
_ = axs[2].hist2d(e_v[:, 1], e_v[:, 2])

In [ ]:
N_SAMPLES = 10000
U = np.random.uniform(size=(3, 3))
a1, a2 = np.random.uniform(size=(2, 1))
a1 = 10
s1, s2 = np.random.normal(size=(2, 3, N_SAMPLES))
e_v = np.sqrt(a1) * U @ s1 + np.sqrt(a2) * np.identity(3) @ s2
e_v = e_v.T
fig, axs = plt.subplots(3, 1, figsize=(20, 60))

_ = axs[0].hist2d(e_v[:, 0], e_v[:, 1])
_ = axs[1].hist2d(e_v[:, 0], e_v[:, 2])
_ = axs[2].hist2d(e_v[:, 1], e_v[:, 2])

In [ ]:
U = np.linalg.inv(U)
e_v = np.sqrt(a1) * U @ s1 + np.sqrt(a2) * np.identity(3) @ s2
e_v = e_v.T
fig, axs = plt.subplots(3, 1, figsize=(20, 60))
_ = axs[0].hist2d(e_v[:, 0], e_v[:, 1])
_ = axs[1].hist2d(e_v[:, 0], e_v[:, 2])
_ = axs[2].hist2d(e_v[:, 1], e_v[:, 2])

In [ ]:
e_vs, e_vs_sigmas, e_vs_valid_hits = [], [], []
for batch in islice(val_data, 0, 1):
    mask = torch.zeros(batch.track_vector.shape[:-1])
    for i, n in enumerate(batch.n_tracks):
        mask[i, :n] = 1
        
    mask = mask.to(DEVICE)
    track_vector = batch.track_vector.to(DEVICE)
    for i in tqdm(range(10000)):
        noise, pred_sigmas, valid_hits = model.get_noise(track_vector, mask)
        noise = np.array(noise.detach().cpu().numpy())
        pred_sigmas = np.array(pred_sigmas.detach().cpu().numpy())
        valid_hits = np.array(valid_hits.detach().cpu().numpy())
        e_vs.append(noise)
        e_vs_sigmas.append(pred_sigmas)
        e_vs_valid_hits.append(valid_hits)

In [ ]:
e_vs_valid_hits = np.stack(e_vs_valid_hits, axis=0)
e_vs_sigmas = np.stack(e_vs_sigmas, axis=0)
e_vs = np.stack(e_vs, axis=0)

In [ ]:
e_vs.shape

In [ ]:
e_vs.shape

In [ ]:
_ = plt.hist2d(e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+0], e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+2])

In [ ]:
_ = plt.hist2d(e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+1], e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+2])

In [ ]:
HIT_INDEX = 1
BATCH_INDEX = 0
TRACK_INDEX = 0

In [ ]:
_ = plt.hist2d(e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+0], e_vs[:, BATCH_INDEX, TRACK_INDEX, 3*HIT_INDEX+1])

In [ ]:
prob = f(p, e_vs_sigmas[0, BATCH_INDEX, TRACK_INDEX, HIT_INDEX])
plt.contour(X, Y, prob, levels=100)